<a href="https://colab.research.google.com/github/malaxovakatyunya/python-ai-Malahova-Ekaterina/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных о статуях мира

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных с помощью pandas.

**Данные:**
- `World_Statues.csv` — информация о памятниках и статуях мира: название, тип, высота, год создания, страна и материал

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файл в pandas DataFrame
3. Очищаем и анализируем структуру столбцов
4. Просматриваем первые строки и выполняем быструю валидацию данных

## 🐱 [1] Клонируем репозиторий курса в Colab

In [2]:
# 🐱 Шаг 1. Клонируем ваш репозиторий в Colab

import os

if not os.path.exists("python-ai-Malahova-Ekaterina"):
    !git clone -q https://github.com/malaxovakatyunya/python-ai-Malahova-Ekaterina.git

%cd python-ai-Malahova-Ekaterina

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Malahova-Ekaterina")

/content/python-ai-Malahova-Ekaterina/python-ai-Malahova-Ekaterina
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Malahova-Ekaterina


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [11]:
# 🐱 Шаг 2A. Чтение CSV-файлов в pandas

import pandas as pd

df_stat = pd.read_csv("data/World_Statues.csv")

print("✅ Загружено строк в df_cost:", len(df_stat))


✅ Загружено строк в df_cost: 1567


## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбцы `statue` и `statueType` содержат URL Wikidata (ссылки на объекты) — они не нужны для анализа.
- Столбцы `statueLabel`, `statueTypeLabel`, `countryLabel`, `materialLabel` содержат читаемые подписи (название статуи, тип, страна, материал).

В этом шаге мы:
- удалим столбцы с URL Wikidata (`statue`, `statueType`);
- переименуем столбцы с суффиксом `Label`, убрав его:
  - `statueLabel` → `statue`
  - `statueTypeLabel` → `statueType`
  - `countryLabel` → `country`
  - `materialLabel` → `material`
- приведём числовые столбцы (`height`, `creationYear`) к корректным типам:
  - `height` → `float` (высота может содержать дробную часть)
  - `creationYear` → `int` (год — целое число)

При приведении к числам мы используем:
- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0;
- `astype(...)` — переводит столбец к нужному типу (`float` или `int`).

> ⚠️ **Важно:** этот шаг обязателен, чтобы избавиться от технических идентификаторов Викиданных и получить удобную таблицу для анализа статуй.

In [18]:
# 🧹 Шаг 2B. Очистка и переименование столбцов

# Удаляем столбцы с URL Wikidata
df_statues = df_stat.drop(columns=["statue", "statueType"])

# Переименовываем столбцы с суффиксом Label
df_statues = df_stat.rename(columns={
    "statueLabel": "statue",
    "statueTypeLabel": "statueType",
    "countryLabel": "country",
    "materialLabel": "material"
})

# Приводим числовые столбцы к корректным типам
df_statues["height"] = pd.to_numeric(
    df_statues["height"], errors="coerce"
).fillna(0).astype(float)

df_statues["creationYear"] = pd.to_numeric(
    df_statues["creationYear"], errors="coerce"
).fillna(0).astype(int)

print("✅ Данные очищены и готовы к анализу")
print("\n📋 Итоговая структура датасета:")
print(df_statues.dtypes)
display(df_stat.head(5))

✅ Данные очищены и готовы к анализу

📋 Итоговая структура датасета:
statue           object
statue           object
statueType       object
statueType       object
height          float64
creationYear      int64
country          object
material         object
dtype: object


,statue,statueLabel,statueType,statueTypeLabel,height,creationYear,countryLabel,materialLabel
0,http://www.wikidata.org/entity/Q646012,Памятник Нидервальд,http://www.wikidata.org/entity/Q575759,военный мемориал,38.18,1883.0,Германия,камень
1,http://www.wikidata.org/entity/Q646012,Памятник Нидервальд,http://www.wikidata.org/entity/Q575759,военный мемориал,38.18,1883.0,Германия,бронза
2,http://www.wikidata.org/entity/Q79961,статуя Христа-Искупителя,http://www.wikidata.org/entity/Q29168169,колоссальная статуя Иисуса Христа,8.00,1920.0,Бразилия,железобетон
3,http://www.wikidata.org/entity/Q79961,статуя Христа-Искупителя,http://www.wikidata.org/entity/Q29168169,колоссальная статуя Иисуса Христа,8.00,1920.0,Бразилия,талькохлорит
4,http://www.wikidata.org/entity/Q79961,статуя Христа-Искупителя,http://www.wikidata.org/entity/Q179700,статуя,30.10,1920.0,Бразилия,железобетон


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор обоих DataFrame:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по бюджету (`capital_cost`).

Для удобства напишем маленькую функцию `show_info(df, name)`, чтобы не повторять один и тот же код два раза.

In [ ]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных

show_info(df_cost, "Бюджеты мультфильмов (df_cost)")
show_info(df_genre, "Жанры, страны и продолжительность (df_genre)")

print("\n📈 Статистика по бюджету (capital_cost):")
print(df_cost["capital_cost"].describe())


📊 Бюджеты мультфильмов (df_cost)
Размер: (421, 2)
Столбцы: film, capital_cost

Первые строки:
            film  capital_cost
0     Покахонтас      55000000
1   Трансформеры       6000000
2            Рио      90000000
3    Огонь и лёд       1200000
4  Цыплёнок Цыпа     150000000

📊 Жанры, страны и продолжительность (df_genre)
Размер: (8011, 4)
Столбцы: film, genre, country, duration

Первые строки:
                                              film                genre  \
0  Паутина Шарлотты 2: Великое приключение Уилбура    музыкальный фильм   
1  Паутина Шарлотты 2: Великое приключение Уилбура     комедийная драма   
2                           Рождественская история  драматический фильм   
3                           Рождественская история    фэнтезийный фильм   
4                           Рождественская история           мультфильм   

  country  duration  
0     США        79  
1     США        79  
2     США        96  
3     США        96  
4     США        96  

📈 Статистика 

In [ ]:
# То же самое, но в миллионах долларов (млн $)
stats_millions = (df_cost["capital_cost"] / 1e6).describe().round(2)
print("\n📈 Статистика по бюджету в млн $:")
print(stats_millions)


📈 Статистика по бюджету в млн $:
count     421.00
mean       70.66
std       165.69
min         0.00
25%         5.50
50%        30.00
75%        90.00
max      2800.00
Name: capital_cost, dtype: float64


## ✅ [4] Быстрая проверка и валидация данных

Здесь мы посмотрим:

- сколько **уникальных** фильмов, стран и жанров есть в данных;
- **какие страны встречаются чаще всего** (Топ‑5 по числу строк);
- **какие жанры самые популярные** (Топ‑10 по числу строк).

Функция `value_counts()`:
- считает, сколько раз каждое значение встречается в столбце;
- сортирует результаты по убыванию.

Метод `.head()` берёт первые N строк, поэтому  
`df_genre["country"].value_counts().head()` даёт **Топ‑5 стран по числу записей**.

In [ ]:
# ✅ Шаг 4. Быстрая проверка и валидация данных

print("🔍 Быстрая проверка данных")

# Датасет 1: бюджеты
print("\nУникальных мультфильмов в df_cost:", df_cost["film"].nunique())
print("Диапазон бюджетов (млн $):",
      df_cost["capital_cost"].min() / 1e6, "—", df_cost["capital_cost"].max() / 1e6)

# Датасет 2: жанры, страны, длительность
print("\nУникальных мультфильмов в df_genre:", df_genre["film"].nunique())
print("Уникальных стран:", df_genre["country"].nunique())
print("Уникальных жанров:", df_genre["genre"].nunique())

print("\nТоп-5 стран по числу записей:")
print(df_genre["country"].value_counts().head())

print("\nТоп-10 жанров:")
print(df_genre["genre"].value_counts().head(10))

🔍 Быстрая проверка данных

Уникальных мультфильмов в df_cost: 417
Диапазон бюджетов (млн $): 9e-06 — 2800.0

Уникальных мультфильмов в df_genre: 2273
Уникальных стран: 86
Уникальных жанров: 240

Топ-5 стран по числу записей:
country
США        2836
Франция     527
СССР        488
Дания       338
Россия      334
Name: count, dtype: int64

Топ-10 жанров:
genre
приключенческий фильм          970
комедийный фильм               833
фэнтезийный фильм              778
семейный фильм                 735
детский фильм                  627
мультфильм                     456
музыкальный фильм              304
драматический фильм            256
боевик                         249
научно-фантастический фильм    220
Name: count, dtype: int64


## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Посмотрели базовую статистику по бюджету (`capital_cost`)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨